In [ ]:
exec(open('init_notebook.py').read())
import random
import carla
import numpy as np
import cv2
import pandas as pd
from agents.vehicles.VehicleFactory import VehicleFactory

In [ ]:
host = '127.0.0.1'
port = 2000
data_dir = 'data/'

tracksMeta = '01_tracksMeta.csv'
tracks = '01_tracks.csv'
recordingMeta = '01_recordingMeta.csv'
image = '01_highway.png'

In [ ]:
client = carla.Client(host, port)
client.set_timeout(5.0)
print(f"Client carla version: {client.get_client_version()}")
print(f"Server carla version: {client.get_server_version()}")
# getting world
world = client.get_world()
debug = world.debug
_map = world.get_map()

spectator = world.get_spectator()
# print(spectator.get_transform())
spectator.set_transform(carla.Transform(carla.Location(x=0, y=0, z=200), carla.Rotation(pitch=-90, yaw=-90)))

In [ ]:
img = cv2.imread(data_dir+image)
tracksMeta = pd.read_csv(data_dir+tracksMeta)
tracks = pd.read_csv(data_dir+tracks)
recordingMeta = pd.read_csv(data_dir+recordingMeta)
print(img.shape)
tracks.tail()

In [ ]:
factor = 0.40424   # 4 * 0.10106 = 0.40424
life_time = 20
thickness = 0.3
pivot = world.get_map().get_spawn_points()[0]
pivot = world.ground_projection(pivot.location, search_distance=400).location

upperLaneMarking = recordingMeta['upperLaneMarkings'].values[0].split(';')
lowerLaneMarking = recordingMeta['lowerLaneMarkings'].values[0].split(';')
upperLaneMarking = [float(x) for x in upperLaneMarking]
lowerLaneMarking = [float(x) for x in lowerLaneMarking]
print(upperLaneMarking)
print(lowerLaneMarking)
print(pivot)
factored_image_size = np.asarray(img.shape) * factor
print(factored_image_size)

In [ ]:
first_line_start = pivot
first_line_end = carla.Location(first_line_start.x+factored_image_size[1], first_line_start.y, first_line_start.z)

left_side_line_start = pivot
left_side_line_end = carla.Location(left_side_line_start.x, left_side_line_start.y+factored_image_size[0], left_side_line_start.z)

bottom_line_start = left_side_line_end
bottom_line_end = carla.Location(bottom_line_start.x+factored_image_size[1], bottom_line_start.y, bottom_line_start.z)

right_side_line_start = first_line_end
right_side_line_end = bottom_line_end

upper_line_1_start = carla.Location(pivot.x, pivot.y + upperLaneMarking[0], pivot.z)
upper_line_1_end = carla.Location(upper_line_1_start.x+factored_image_size[1], upper_line_1_start.y, upper_line_1_start.z)

upper_line_2_start = carla.Location(pivot.x, pivot.y + upperLaneMarking[1], pivot.z)
upper_line_2_end = carla.Location(upper_line_2_start.x+factored_image_size[1], upper_line_2_start.y, upper_line_2_start.z)

upper_line_3_start = carla.Location(pivot.x, pivot.y + upperLaneMarking[2], pivot.z)
upper_line_3_end = carla.Location(upper_line_3_start.x+factored_image_size[1], upper_line_3_start.y, upper_line_3_start.z)

lower_line_1_start = carla.Location(pivot.x, pivot.y + lowerLaneMarking[0], pivot.z)
lower_line_1_end = carla.Location(lower_line_1_start.x+factored_image_size[1], lower_line_1_start.y, lower_line_1_start.z)

lower_line_2_start = carla.Location(pivot.x, pivot.y + lowerLaneMarking[1], pivot.z)
lower_line_2_end = carla.Location(lower_line_2_start.x+factored_image_size[1], lower_line_2_start.y, lower_line_2_start.z)

lower_line_3_start = carla.Location(pivot.x, pivot.y + lowerLaneMarking[2], pivot.z)
lower_line_3_end = carla.Location(lower_line_3_start.x+factored_image_size[1], lower_line_3_start.y, lower_line_3_start.z)



In [ ]:
debug.draw_line(begin=first_line_start,
                end=first_line_end,
                thickness=thickness,
                color=carla.Color(255,0,0),
                life_time=life_time)

debug.draw_line(begin=left_side_line_start,
                end=left_side_line_end,
                thickness=thickness,
                color=carla.Color(255,0,0),
                life_time=life_time)

debug.draw_line(begin=bottom_line_start,
                end=bottom_line_end,
                thickness=thickness,
                color=carla.Color(255,0,0),
                life_time=life_time)

debug.draw_line(begin=right_side_line_start,
                end=right_side_line_end,
                thickness=thickness,
                color=carla.Color(255,0,0),
                life_time=life_time)

debug.draw_line(begin=upper_line_1_start,
                end=upper_line_1_end,
                thickness=thickness,
                color=carla.Color(0,0,255),
                life_time=life_time)

debug.draw_line(begin=upper_line_2_start,
                end=upper_line_2_end,
                thickness=thickness,
                color=carla.Color(0,0,255),
                life_time=life_time)

debug.draw_line(begin=upper_line_3_start,
                end=upper_line_3_end,
                thickness=thickness,
                color=carla.Color(0,0,255),
                life_time=life_time)

debug.draw_line(begin=lower_line_1_start,
                end=lower_line_1_end,
                thickness=thickness,
                color=carla.Color(0,0,255),
                life_time=life_time)

debug.draw_line(begin=lower_line_2_start,
                end=lower_line_2_end,
                thickness=thickness,
                color=carla.Color(0,0,255),
                life_time=life_time)

debug.draw_line(begin=lower_line_3_start,
                end=lower_line_3_end,
                thickness=thickness,
                color=carla.Color(0,0,255),
                life_time=life_time)



In [ ]:
from lib.utils import Utils


def origin_transform_to_pivot(pivot, point, height=0):
    new_x = pivot.x + point.x
    new_y = pivot.y + point.y
    new_z = height
    return carla.Location(x=new_x, y=new_y, z=new_z)

def draw_transform_list(debug, transforms, life_time):
    for transform in transforms:
        debug.draw_point(transform.location, size=0.1, color=carla.Color(255,0,0), life_time=life_time)
    pass

agent_id = 1
agent_track_data = tracks[tracks['id'] == agent_id]
trajectory_transforms = []
for index, row in agent_track_data.iterrows():
    point = carla.Location(x=row['x'], y=row['y'], z=0)
    location = origin_transform_to_pivot(pivot, point, 0.5)
    rotation = carla.Rotation(pitch=0, yaw=0, roll=0)
    transform = carla.Transform(location, rotation)
    trajectory_transforms.append(transform)

# print(trajectory_transforms)

draw_transform_list(debug, trajectory_transforms, life_time=10)




In [ ]:
waypoints = []
for transform in trajectory_transforms:
    waypoint = _map.get_waypoint(transform.location)
    print('road id ', waypoint.road_id, 
          'section id ', waypoint.section_id, 
          'lane id ', waypoint.lane_id)
    
    waypoints.append(waypoint)
# print(waypoints)

# Utils.draw_waypoints(debug, waypoints, life_time=10)

In [ ]:
bpLib = world.get_blueprint_library()
vehicles = bpLib.filter('vehicle.*')
vehicle_bp = vehicles[0]

vehicle = world.spawn_actor(vehicle_bp, trajectory_transforms[0])
# vehicle.set_simulate_physics(False)
world.wait_for_tick()


vehicle_factory = VehicleFactory(world)
basic_agent = vehicle_factory.createAgent(vehicle=vehicle, target_speed=40)

In [ ]:
print(basic_agent._vehicle.get_transform())
# print(trajectory_transforms[-1])


In [ ]:
basic_agent.set_destination(trajectory_transforms[-1].location)

In [ ]:
while basic_agent.done() == False:
    control = basic_agent.run_step()
    basic_agent.vehicle.apply_control(control)
    world.wait_for_tick()

In [ ]:
basic_agent.destroy()